In [ ]:
%pip install -q timm==0.9.5 torch torchvision tqdm matplotlib seaborn scikit-learn

In [ ]:
import os
import torch
import torch.nn as nn
import timm
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from torchvision import transforms
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
from tqdm.auto import tqdm
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import pandas as pd
import random
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler

In [ ]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set random seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

In [ ]:
class ChickenFecesDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = DEFAULT_CLASSES
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        self.samples = []
        
        # Cari semua file gambar di direktori data
        for class_name in self.classes:
            class_dir = os.path.join(data_dir, class_name)
            if os.path.isdir(class_dir):
                class_files = [f for f in os.listdir(class_dir) 
                              if os.path.isfile(os.path.join(class_dir, f)) and 
                              f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                print(f"Kelas {class_name}: {len(class_files)} gambar ditemukan")
                
                for img_name in class_files:
                    img_path = os.path.join(class_dir, img_name)
                    self.samples.append((img_path, self.class_to_idx[class_name]))
        
        print(f"Total dataset: {len(self.samples)} gambar")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        try:
            # Buka gambar dan konversi ke RGB
            image = Image.open(img_path).convert('RGB')
            
            # Terapkan transformasi jika ada
            if self.transform:
                image = self.transform(image)
            
            return image, label
                
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            dummy_img = torch.zeros((3, IMAGE_SIZE, IMAGE_SIZE))
            return dummy_img, label

In [ ]:
def get_transforms(train=True):
    # Nilai normalisasi ImageNet
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    
    if train:
        # Transformasi training dengan augmentasi
        return transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ])
    else:
        # Transformasi validasi (tanpa augmentasi)
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ])

In [ ]:
def create_dataloaders(dataset_dir, batch_size=16):
    # Path untuk dataset train dan test
    train_dir = os.path.join(dataset_dir, 'train')
    test_dir = os.path.join(dataset_dir, 'test')
    
    # Buat datasets
    train_dataset = ChickenFecesDataset(
        train_dir,
        transform=get_transforms(train=True)
    )
    
    test_dataset = ChickenFecesDataset(
        test_dir,
        transform=get_transforms(train=False)
    )
    
    # Buat dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    
    return train_loader, val_loader

In [ ]:
class ChickenFecesClassifier(nn.Module):
    def __init__(self, num_classes=4):
        super(ChickenFecesClassifier, self).__init__()
        
        # Gunakan ViT pre-trained
        self.model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)
        
        # Dapatkan in_features dari head
        in_features = self.model.head.in_features
        
        # Ganti head dengan custom classifier untuk fine-tuning
        self.model.head = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        return self.model(x)

In [ ]:
def train_model(model, train_loader, val_loader, num_epochs=20, learning_rate=0.0005):
    # Buat direktori untuk menyimpan model
    os.makedirs('models', exist_ok=True)
    
    # Definisikan criterion, optimizer, dan scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
    
    # Variabel untuk tracking
    best_val_acc = 0.0
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    
    # Loop training
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        # Progress bar untuk training
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
        
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass dan optimize
            loss.backward()
            optimizer.step()
            
            # Statistik
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            
            # Update progress bar
            train_pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100 * train_correct / train_total:.2f}%'
            })
        
        # Hitung rata-rata loss dan accuracy
        epoch_train_loss = train_loss / len(train_loader.dataset)
        epoch_train_acc = 100 * train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        all_preds = []
        all_labels = []
        
        # Progress bar untuk validation
        val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")
        
        with torch.no_grad():
            for inputs, labels in val_pbar:
                inputs, labels = inputs.to(device), labels.to(device)
                
                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                # Statistik
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                
                # Collect predictions untuk confusion matrix
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                
                # Update progress bar
                val_pbar.set_postfix({
                    'loss': f'{loss.item():.4f}',
                    'acc': f'{100 * val_correct / val_total:.2f}%'
                })
                
        # Hitung rata-rata loss dan accuracy
        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_val_acc = 100 * val_correct / val_total
        
        # Update LR scheduler
        scheduler.step(epoch_val_loss)
        
        # Simpan metrics
        history['train_loss'].append(epoch_train_loss)
        history['val_loss'].append(epoch_val_loss)
        history['train_acc'].append(epoch_train_acc)
        history['val_acc'].append(epoch_val_acc)
        
        # Print summary
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%")
        print(f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.2f}%")
        
        # Simpan model terbaik
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            torch.save(model.state_dict(), 'models/best_vit_chicken_classifier.pth')
            print(f"Model disimpan dengan akurasi: {best_val_acc:.2f}%")
            
            # Buat confusion matrix ketika model terbaik
            cm = confusion_matrix(all_labels, all_preds)
            plt.figure(figsize=(10, 8))
            display_classes = [cls.replace('Chicken_', '') for cls in DEFAULT_CLASSES]
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                      xticklabels=display_classes, yticklabels=display_classes)
            plt.xlabel('Diprediksi')
            plt.ylabel('Aktual')
            plt.title(f'Confusion Matrix - Epoch {epoch+1} (Acc: {epoch_val_acc:.2f}%)')
            plt.tight_layout()
            plt.savefig(f'confusion_matrix_epoch_{epoch+1}.png')
            plt.close()
    
    # Plot training history
    plt.figure(figsize=(12, 5))
    
    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training dan Validation Loss')
    plt.legend()
    
    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training dan Validation Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()
    
    return history, best_val_acc

In [ ]:
def predict_image(model, image_path):
    # Transformasi untuk inference
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    # Load dan preprocess gambar
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Set model ke mode evaluasi
    model.eval()
    
    # Predict
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs, 1)[0]
        predicted_class = torch.argmax(probabilities).item()
    
    # Dapatkan nama kelas dan confidence
    class_name = DEFAULT_CLASSES[predicted_class]
    confidence = probabilities[predicted_class].item() * 100
    
    # Dapatkan semua skor
    all_scores = [(DEFAULT_CLASSES[i], prob.item() * 100) for i, prob in enumerate(probabilities)]
    all_scores.sort(key=lambda x: x[1], reverse=True)
    
    # Visualisasi
    plt.figure(figsize=(12, 5))
    
    # Tampilkan gambar
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title(f"Prediksi: {class_name.replace('Chicken_', '')}\nConfidence: {confidence:.2f}%")
    plt.axis('off')
    
    # Tampilkan bar chart
    plt.subplot(1, 2, 2)
    class_names = [cls.replace('Chicken_', '') for cls, _ in all_scores]
    scores = [score for _, score in all_scores]
    
    bars = plt.barh(class_names, scores, color='skyblue')
    bars[0].set_color('navy')  # Highlight kelas tertinggi
    
    for i, v in enumerate(scores):
        plt.text(v + 1, i, f"{v:.1f}%", va='center')
    
    plt.xlabel('Confidence (%)')
    plt.title('Probabilitas Kelas')
    plt.xlim(0, 105)
    
    plt.tight_layout()
    plt.savefig('prediction_result.png')
    plt.show()
    
    # Informasi penyakit
    disease_info = get_disease_info(class_name)
    print(f"\nInformasi Penyakit:")
    print(disease_info['description'])
    print("\nRekomendasi:")
    for i, rec in enumerate(disease_info['recommendations']):
        print(f"{i+1}. {rec}")
    
    return class_name, confidence, all_scores

In [ ]:
def main():
    # Path ke dataset
    dataset_dir = 'chicken_feces_dataset'  # Sesuaikan dengan path di sistem Anda
    
    # Hyperparameters
    batch_size = 16
    num_epochs = 20
    learning_rate = 0.0005
    
    # Buat dataloader
    train_loader, val_loader = create_dataloaders(dataset_dir, batch_size)
    
    # Buat model
    model = ChickenFecesClassifier(num_classes=len(DEFAULT_CLASSES))
    model = model.to(device)
    
    # Train model
    print("Memulai training...")
    history, best_acc = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=num_epochs,
        learning_rate=learning_rate
    )
    
    print(f"Training selesai! Akurasi terbaik: {best_acc:.2f}%")
    
    return model, history, best_acc

# Run training
if __name__ == "__main__":
    model, history, best_acc = main()

In [ ]:
def get_disease_info(class_name):
    """Menyediakan informasi tentang penyakit dan rekomendasi"""
    
    info = {
        'Chicken_Healthy': {
            'description': 'Kotoran menunjukkan ayam dalam kondisi sehat.',
            'recommendations': [
                'Pertahankan kualitas pakan dan manajemen peternakan yang baik',
                'Lanjutkan program vaksinasi secara rutin',
                'Pantau kondisi kotoran ayam secara berkala'
            ]
        },
        'Chicken_Coccidiosis': {
            'description': 'Coccidiosis adalah penyakit parasit yang disebabkan oleh protozoa Eimeria, ditandai dengan kotoran berdarah atau berwarna kemerahan.',
            'recommendations': [
                'Berikan pengobatan anticoccidial sesuai resep dokter hewan',
                'Jaga kebersihan kandang, hindari kelembaban berlebih',
                'Isolasi ayam yang terinfeksi untuk mencegah penyebaran',
                'Tingkatkan sanitasi dan desinfeksi peralatan peternakan'
            ]
        },
        'Chicken_NewCastleDisease': {
            'description': 'Newcastle Disease (ND) adalah penyakit virus yang sangat menular dengan gejala kotoran berwarna hijau atau putih berair.',
            'recommendations': [
                'Segera konsultasikan dengan dokter hewan',
                'Lakukan vaksinasi ND pada seluruh ayam di peternakan',
                'Isolasi ayam yang terinfeksi dengan ketat',
                'Lakukan desinfeksi menyeluruh pada kandang dan peralatan',
                'Pantau kondisi ayam lain untuk gejala awal'
            ]
        },
        'Chicken_Salmonella': {
            'description': 'Salmonellosis disebabkan oleh bakteri Salmonella dengan kotoran berwarna kekuningan atau putih kapur.',
            'recommendations': [
                'Berikan antibiotik sesuai resep dokter hewan',
                'Tingkatkan biosecurity di peternakan',
                'Jaga kebersihan sumber air, pakan, dan peralatan',
                'Lakukan pemeriksaan rutin untuk Salmonella pada semua ayam',
                'Isolasi ayam yang terinfeksi dan tangani secara hati-hati'
            ]
        }
    }
    
    return info.get(class_name, {
        'description': 'Informasi tidak tersedia untuk jenis kotoran ini',
        'recommendations': ['Konsultasikan dengan dokter hewan untuk diagnosa lebih lanjut']
    })

# Kode untuk load model dan melakukan prediksi
def load_model_and_predict(model_path, image_path):
    # Buat model
    model = ChickenFecesClassifier(num_classes=len(DEFAULT_CLASSES))
    
    # Load model weights
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    
    # Prediksi
    class_name, confidence, all_scores = predict_image(model, image_path)
    
    return class_name, confidence, all_scores

# Contoh penggunaan:
# model_path = 'models/best_vit_chicken_classifier.pth'
# image_path = 'path/to/test_image.jpg'
# class_name, confidence, all_scores = load_model_and_predict(model_path, image_path)

In [ ]:
%pip install -q timm==0.9.5 torch torchvision tqdm matplotlib seaborn scikit-learn

In [ ]:
import os
import torch
import torch.nn as nn
import timm
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from torchvision import transforms
from sklearn.metrics import confusion_matrix
from tqdm.auto import tqdm
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Periksa ketersediaan GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Menggunakan device: {device}")

# Tetapkan kelas untuk dataset
DEFAULT_CLASSES = ["Chicken_Coccidiosis", "Chicken_Healthy", "Chicken_NewCastleDisease", "Chicken_Salmonella"]
IMAGE_SIZE = 224  # Ukuran gambar untuk ViT

In [ ]:
class ChickenFecesDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = DEFAULT_CLASSES
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        self.samples = []
        
        # Cari semua file gambar di direktori data
        for class_name in self.classes:
            class_dir = os.path.join(data_dir, class_name)
            if os.path.isdir(class_dir):
                class_files = [f for f in os.listdir(class_dir) 
                              if os.path.isfile(os.path.join(class_dir, f)) and 
                              f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                print(f"Kelas {class_name}: {len(class_files)} gambar ditemukan")
                
                for img_name in class_files:
                    img_path = os.path.join(class_dir, img_name)
                    self.samples.append((img_path, self.class_to_idx[class_name]))
        
        print(f"Total dataset: {len(self.samples)} gambar")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        try:
            # Buka gambar dan konversi ke RGB
            image = Image.open(img_path).convert('RGB')
            
            # Terapkan transformasi jika ada
            if self.transform:
                image = self.transform(image)
            
            return image, label
                
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            dummy_img = torch.zeros((3, IMAGE_SIZE, IMAGE_SIZE))
            return dummy_img, label

In [ ]:
def get_transforms(train=True):
    # Nilai normalisasi ImageNet
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    
    if train:
        # Transformasi training dengan augmentasi
        return transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ])
    else:
        # Transformasi validasi (tanpa augmentasi)
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ])

In [ ]:
def create_dataloaders(dataset_dir, batch_size=16):
    # Path untuk dataset train dan test
    train_dir = os.path.join(dataset_dir, 'train')
    test_dir = os.path.join(dataset_dir, 'test')
    
    # Buat datasets
    train_dataset = ChickenFecesDataset(
        train_dir,
        transform=get_transforms(train=True)
    )
    
    test_dataset = ChickenFecesDataset(
        test_dir,
        transform=get_transforms(train=False)
    )
    
    # Buat dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    
    return train_loader, val_loader

In [ ]:
class ChickenFecesClassifier(nn.Module):
    def __init__(self, num_classes=4):
        super(ChickenFecesClassifier, self).__init__()
        
        # Gunakan ViT pre-trained
        self.model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)
        
        # Dapatkan in_features dari head
        in_features = self.model.head.in_features
        
        # Ganti head dengan custom classifier untuk fine-tuning
        self.model.head = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        return self.model(x)

In [ ]:
def train_model(model, train_loader, val_loader, num_epochs=20, learning_rate=0.0005):
    # Buat direktori untuk menyimpan model
    os.makedirs('models', exist_ok=True)
    
    # Definisikan criterion, optimizer, dan scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
    
    # Variabel untuk tracking
    best_val_acc = 0.0
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    
    # Loop training
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        # Progress bar untuk training
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
        
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass dan optimize
            loss.backward()
            optimizer.step()
            
            # Statistik
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            
            # Update progress bar
            train_pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100 * train_correct / train_total:.2f}%'
            })
        
        # Hitung rata-rata loss dan accuracy
        epoch_train_loss = train_loss / len(train_loader.dataset)
        epoch_train_acc = 100 * train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        all_preds = []
        all_labels = []
        
        # Progress bar untuk validation
        val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")
        
        with torch.no_grad():
            for inputs, labels in val_pbar:
                inputs, labels = inputs.to(device), labels.to(device)
                
                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                # Statistik
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                
                # Collect predictions untuk confusion matrix
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                
                # Update progress bar
                val_pbar.set_postfix({
                    'loss': f'{loss.item():.4f}',
                    'acc': f'{100 * val_correct / val_total:.2f}%'
                })
                
        # Hitung rata-rata loss dan accuracy
        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_val_acc = 100 * val_correct / val_total
        
        # Update LR scheduler
        scheduler.step(epoch_val_loss)
        
        # Simpan metrics
        history['train_loss'].append(epoch_train_loss)
        history['val_loss'].append(epoch_val_loss)
        history['train_acc'].append(epoch_train_acc)
        history['val_acc'].append(epoch_val_acc)
        
        # Print summary
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%")
        print(f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.2f}%")
        
        # Simpan model terbaik
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            torch.save(model.state_dict(), 'models/best_vit_chicken_classifier.pth')
            print(f"Model disimpan dengan akurasi: {best_val_acc:.2f}%")
            
            # Buat confusion matrix ketika model terbaik
            cm = confusion_matrix(all_labels, all_preds)
            plt.figure(figsize=(10, 8))
            display_classes = [cls.replace('Chicken_', '') for cls in DEFAULT_CLASSES]
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                      xticklabels=display_classes, yticklabels=display_classes)
            plt.xlabel('Diprediksi')
            plt.ylabel('Aktual')
            plt.title(f'Confusion Matrix - Epoch {epoch+1} (Acc: {epoch_val_acc:.2f}%)')
            plt.tight_layout()
            plt.savefig(f'confusion_matrix_epoch_{epoch+1}.png')
            plt.close()
    
    # Plot training history
    plt.figure(figsize=(12, 5))
    
    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training dan Validation Loss')
    plt.legend()
    
    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training dan Validation Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()
    
    return history, best_val_acc

In [ ]:
def predict_image(model, image_path):
    # Transformasi untuk inference
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    # Load dan preprocess gambar
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Set model ke mode evaluasi
    model.eval()
    
    # Predict
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs, 1)[0]
        predicted_class = torch.argmax(probabilities).item()
    
    # Dapatkan nama kelas dan confidence
    class_name = DEFAULT_CLASSES[predicted_class]
    confidence = probabilities[predicted_class].item() * 100
    
    # Dapatkan semua skor
    all_scores = [(DEFAULT_CLASSES[i], prob.item() * 100) for i, prob in enumerate(probabilities)]
    all_scores.sort(key=lambda x: x[1], reverse=True)
    
    # Visualisasi
    plt.figure(figsize=(12, 5))
    
    # Tampilkan gambar
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title(f"Prediksi: {class_name.replace('Chicken_', '')}\nConfidence: {confidence:.2f}%")
    plt.axis('off')
    
    # Tampilkan bar chart
    plt.subplot(1, 2, 2)
    class_names = [cls.replace('Chicken_', '') for cls, _ in all_scores]
    scores = [score for _, score in all_scores]
    
    bars = plt.barh(class_names, scores, color='skyblue')
    bars[0].set_color('navy')  # Highlight kelas tertinggi
    
    for i, v in enumerate(scores):
        plt.text(v + 1, i, f"{v:.1f}%", va='center')
    
    plt.xlabel('Confidence (%)')
    plt.title('Probabilitas Kelas')
    plt.xlim(0, 105)
    
    plt.tight_layout()
    plt.savefig('prediction_result.png')
    plt.show()
    
    # Informasi penyakit
    disease_info = get_disease_info(class_name)
    print(f"\nInformasi Penyakit:")
    print(disease_info['description'])
    print("\nRekomendasi:")
    for i, rec in enumerate(disease_info['recommendations']):
        print(f"{i+1}. {rec}")
    
    return class_name, confidence, all_scores

In [ ]:
def main():
    # Path ke dataset
    dataset_dir = 'chicken_feces_dataset'  # Sesuaikan dengan path di sistem Anda
    
    # Hyperparameters
    batch_size = 16
    num_epochs = 20
    learning_rate = 0.0005
    
    # Buat dataloader
    train_loader, val_loader = create_dataloaders(dataset_dir, batch_size)
    
    # Buat model
    model = ChickenFecesClassifier(num_classes=len(DEFAULT_CLASSES))
    model = model.to(device)
    
    # Train model
    print("Memulai training...")
    history, best_acc = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=num_epochs,
        learning_rate=learning_rate
    )
    
    print(f"Training selesai! Akurasi terbaik: {best_acc:.2f}%")
    
    return model, history, best_acc

# Run training
if __name__ == "__main__":
    model, history, best_acc = main()

In [ ]:
def get_disease_info(class_name):
    """Menyediakan informasi tentang penyakit dan rekomendasi"""
    
    info = {
        'Chicken_Healthy': {
            'description': 'Kotoran menunjukkan ayam dalam kondisi sehat.',
            'recommendations': [
                'Pertahankan kualitas pakan dan manajemen peternakan yang baik',
                'Lanjutkan program vaksinasi secara rutin',
                'Pantau kondisi kotoran ayam secara berkala'
            ]
        },
        'Chicken_Coccidiosis': {
            'description': 'Coccidiosis adalah penyakit parasit yang disebabkan oleh protozoa Eimeria, ditandai dengan kotoran berdarah atau berwarna kemerahan.',
            'recommendations': [
                'Berikan pengobatan anticoccidial sesuai resep dokter hewan',
                'Jaga kebersihan kandang, hindari kelembaban berlebih',
                'Isolasi ayam yang terinfeksi untuk mencegah penyebaran',
                'Tingkatkan sanitasi dan desinfeksi peralatan peternakan'
            ]
        },
        'Chicken_NewCastleDisease': {
            'description': 'Newcastle Disease (ND) adalah penyakit virus yang sangat menular dengan gejala kotoran berwarna hijau atau putih berair.',
            'recommendations': [
                'Segera konsultasikan dengan dokter hewan',
                'Lakukan vaksinasi ND pada seluruh ayam di peternakan',
                'Isolasi ayam yang terinfeksi dengan ketat',
                'Lakukan desinfeksi menyeluruh pada kandang dan peralatan',
                'Pantau kondisi ayam lain untuk gejala awal'
            ]
        },
        'Chicken_Salmonella': {
            'description': 'Salmonellosis disebabkan oleh bakteri Salmonella dengan kotoran berwarna kekuningan atau putih kapur.',
            'recommendations': [
                'Berikan antibiotik sesuai resep dokter hewan',
                'Tingkatkan biosecurity di peternakan',
                'Jaga kebersihan sumber air, pakan, dan peralatan',
                'Lakukan pemeriksaan rutin untuk Salmonella pada semua ayam',
                'Isolasi ayam yang terinfeksi dan tangani secara hati-hati'
            ]
        }
    }
    
    return info.get(class_name, {
        'description': 'Informasi tidak tersedia untuk jenis kotoran ini',
        'recommendations': ['Konsultasikan dengan dokter hewan untuk diagnosa lebih lanjut']
    })

# Kode untuk load model dan melakukan prediksi
def load_model_and_predict(model_path, image_path):
    # Buat model
    model = ChickenFecesClassifier(num_classes=len(DEFAULT_CLASSES))
    
    # Load model weights
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    
    # Prediksi
    class_name, confidence, all_scores = predict_image(model, image_path)
    
    return class_name, confidence, all_scores

# Contoh penggunaan:
# model_path = 'models/best_vit_chicken_classifier.pth'
# image_path = 'path/to/test_image.jpg'
# class_name, confidence, all_scores = load_model_and_predict(model_path, image_path)

In [ ]:
class ChickenFecesDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = DEFAULT_CLASSES
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        self.samples = []
        
        # Cari semua file gambar di direktori data
        for class_name in self.classes:
            class_dir = os.path.join(data_dir, class_name)
            if os.path.isdir(class_dir):
                class_files = [f for f in os.listdir(class_dir) 
                              if os.path.isfile(os.path.join(class_dir, f)) and 
                              f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                print(f"Kelas {class_name}: {len(class_files)} gambar ditemukan")
                
                for img_name in class_files:
                    img_path = os.path.join(class_dir, img_name)
                    self.samples.append((img_path, self.class_to_idx[class_name]))
        
        print(f"Total dataset: {len(self.samples)} gambar")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        try:
            # Buka gambar dan konversi ke RGB
            image = Image.open(img_path).convert('RGB')
            
            # Terapkan transformasi jika ada
            if self.transform:
                image = self.transform(image)
            
            return image, label
                
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            dummy_img = torch.zeros((3, IMAGE_SIZE, IMAGE_SIZE))
            return dummy_img, label

In [ ]:
def get_transforms(train=True):
    # Nilai normalisasi ImageNet
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    
    if train:
        # Transformasi training dengan augmentasi
        return transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ])
    else:
        # Transformasi validasi (tanpa augmentasi)
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ])

In [ ]:
def create_dataloaders(dataset_dir, batch_size=16):
    # Path untuk dataset train dan test
    train_dir = os.path.join(dataset_dir, 'train')
    test_dir = os.path.join(dataset_dir, 'test')
    
    # Buat datasets
    train_dataset = ChickenFecesDataset(
        train_dir,
        transform=get_transforms(train=True)
    )
    
    test_dataset = ChickenFecesDataset(
        test_dir,
        transform=get_transforms(train=False)
    )
    
    # Buat dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    
    return train_loader, val_loader

In [ ]:
class ChickenFecesClassifier(nn.Module):
    def __init__(self, num_classes=4):
        super(ChickenFecesClassifier, self).__init__()
        
        # Gunakan ViT pre-trained
        self.model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)
        
        # Dapatkan in_features dari head
        in_features = self.model.head.in_features
        
        # Ganti head dengan custom classifier untuk fine-tuning
        self.model.head = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        return self.model(x)

In [ ]:
def train_model(model, train_loader, val_loader, num_epochs=20, learning_rate=0.0005):
    # Buat direktori untuk menyimpan model
    os.makedirs('models', exist_ok=True)
    
    # Definisikan criterion, optimizer, dan scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
    
    # Variabel untuk tracking
    best_val_acc = 0.0
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    
    # Loop training
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        # Progress bar untuk training
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
        
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass dan optimize
            loss.backward()
            optimizer.step()
            
            # Statistik
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            
            # Update progress bar
            train_pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100 * train_correct / train_total:.2f}%'
            })
        
        # Hitung rata-rata loss dan accuracy
        epoch_train_loss = train_loss / len(train_loader.dataset)
        epoch_train_acc = 100 * train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        all_preds = []
        all_labels = []
        
        # Progress bar untuk validation
        val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")
        
        with torch.no_grad():
            for inputs, labels in val_pbar:
                inputs, labels = inputs.to(device), labels.to(device)
                
                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                # Statistik
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                
                # Collect predictions untuk confusion matrix
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                
                # Update progress bar
                val_pbar.set_postfix({
                    'loss': f'{loss.item():.4f}',
                    'acc': f'{100 * val_correct / val_total:.2f}%'
                })
                
        # Hitung rata-rata loss dan accuracy
        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_val_acc = 100 * val_correct / val_total
        
        # Update LR scheduler
        scheduler.step(epoch_val_loss)
        
        # Simpan metrics
        history['train_loss'].append(epoch_train_loss)
        history['val_loss'].append(epoch_val_loss)
        history['train_acc'].append(epoch_train_acc)
        history['val_acc'].append(epoch_val_acc)
        
        # Print summary
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%")
        print(f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.2f}%")
        
        # Simpan model terbaik
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            torch.save(model.state_dict(), 'models/best_vit_chicken_classifier.pth')
            print(f"Model disimpan dengan akurasi: {best_val_acc:.2f}%")
            
            # Buat confusion matrix ketika model terbaik
            cm = confusion_matrix(all_labels, all_preds)
            plt.figure(figsize=(10, 8))
            display_classes = [cls.replace('Chicken_', '') for cls in DEFAULT_CLASSES]
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                      xticklabels=display_classes, yticklabels=display_classes)
            plt.xlabel('Diprediksi')
            plt.ylabel('Aktual')
            plt.title(f'Confusion Matrix - Epoch {epoch+1} (Acc: {epoch_val_acc:.2f}%)')
            plt.tight_layout()
            plt.savefig(f'confusion_matrix_epoch_{epoch+1}.png')
            plt.close()
    
    # Plot training history
    plt.figure(figsize=(12, 5))
    
    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training dan Validation Loss')
    plt.legend()
    
    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training dan Validation Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()
    
    return history, best_val_acc

In [ ]:
def predict_image(model, image_path):
    # Transformasi untuk inference
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    # Load dan preprocess gambar
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Set model ke mode evaluasi
    model.eval()
    
    # Predict
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs, 1)[0]
        predicted_class = torch.argmax(probabilities).item()
    
    # Dapatkan nama kelas dan confidence
    class_name = DEFAULT_CLASSES[predicted_class]
    confidence = probabilities[predicted_class].item() * 100
    
    # Dapatkan semua skor
    all_scores = [(DEFAULT_CLASSES[i], prob.item() * 100) for i, prob in enumerate(probabilities)]
    all_scores.sort(key=lambda x: x[1], reverse=True)
    
    # Visualisasi
    plt.figure(figsize=(12, 5))
    
    # Tampilkan gambar
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title(f"Prediksi: {class_name.replace('Chicken_', '')}\nConfidence: {confidence:.2f}%")
    plt.axis('off')
    
    # Tampilkan bar chart
    plt.subplot(1, 2, 2)
    class_names = [cls.replace('Chicken_', '') for cls, _ in all_scores]
    scores = [score for _, score in all_scores]
    
    bars = plt.barh(class_names, scores, color='skyblue')
    bars[0].set_color('navy')  # Highlight kelas tertinggi
    
    for i, v in enumerate(scores):
        plt.text(v + 1, i, f"{v:.1f}%", va='center')
    
    plt.xlabel('Confidence (%)')
    plt.title('Probabilitas Kelas')
    plt.xlim(0, 105)
    
    plt.tight_layout()
    plt.savefig('prediction_result.png')
    plt.show()
    
    # Informasi penyakit
    disease_info = get_disease_info(class_name)
    print(f"\nInformasi Penyakit:")
    print(disease_info['description'])
    print("\nRekomendasi:")
    for i, rec in enumerate(disease_info['recommendations']):
        print(f"{i+1}. {rec}")
    
    return class_name, confidence, all_scores

In [ ]:
def main():
    # Path ke dataset
    dataset_dir = 'chicken_feces_dataset'  # Sesuaikan dengan path di sistem Anda
    
    # Hyperparameters
    batch_size = 16
    num_epochs = 20
    learning_rate = 0.0005
    
    # Buat dataloader
    train_loader, val_loader = create_dataloaders(dataset_dir, batch_size)
    
    # Buat model
    model = ChickenFecesClassifier(num_classes=len(DEFAULT_CLASSES))
    model = model.to(device)
    
    # Train model
    print("Memulai training...")
    history, best_acc = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=num_epochs,
        learning_rate=learning_rate
    )
    
    print(f"Training selesai! Akurasi terbaik: {best_acc:.2f}%")
    
    return model, history, best_acc

# Run training
if __name__ == "__main__":
    model, history, best_acc = main()

In [ ]:
def get_disease_info(class_name):
    """Menyediakan informasi tentang penyakit dan rekomendasi"""
    
    info = {
        'Chicken_Healthy': {
            'description': 'Kotoran menunjukkan ayam dalam kondisi sehat.',
            'recommendations': [
                'Pertahankan kualitas pakan dan manajemen peternakan yang baik',
                'Lanjutkan program vaksinasi secara rutin',
                'Pantau kondisi kotoran ayam secara berkala'
            ]
        },
        'Chicken_Coccidiosis': {
            'description': 'Coccidiosis adalah penyakit parasit yang disebabkan oleh protozoa Eimeria, ditandai dengan kotoran berdarah atau berwarna kemerahan.',
            'recommendations': [
                'Berikan pengobatan anticoccidial sesuai resep dokter hewan',
                'Jaga kebersihan kandang, hindari kelembaban berlebih',
                'Isolasi ayam yang terinfeksi untuk mencegah penyebaran',
                'Tingkatkan sanitasi dan desinfeksi peralatan peternakan'
            ]
        },
        'Chicken_NewCastleDisease': {
            'description': 'Newcastle Disease (ND) adalah penyakit virus yang sangat menular dengan gejala kotoran berwarna hijau atau putih berair.',
            'recommendations': [
                'Segera konsultasikan dengan dokter hewan',
                'Lakukan vaksinasi ND pada seluruh ayam di peternakan',
                'Isolasi ayam yang terinfeksi dengan ketat',
                'Lakukan desinfeksi menyeluruh pada kandang dan peralatan',
                'Pantau kondisi ayam lain untuk gejala awal'
            ]
        },
        'Chicken_Salmonella': {
            'description': 'Salmonellosis disebabkan oleh bakteri Salmonella dengan kotoran berwarna kekuningan atau putih kapur.',
            'recommendations': [
                'Berikan antibiotik sesuai resep dokter hewan',
                'Tingkatkan biosecurity di peternakan',
                'Jaga kebersihan sumber air, pakan, dan peralatan',
                'Lakukan pemeriksaan rutin untuk Salmonella pada semua ayam',
                'Isolasi ayam yang terinfeksi dan tangani secara hati-hati'
            ]
        }
    }
    
    return info.get(class_name, {
        'description': 'Informasi tidak tersedia untuk jenis kotoran ini',
        'recommendations': ['Konsultasikan dengan dokter hewan untuk diagnosa lebih lanjut']
    })

# Kode untuk load model dan melakukan prediksi
def load_model_and_predict(model_path, image_path):
    # Buat model
    model = ChickenFecesClassifier(num_classes=len(DEFAULT_CLASSES))
    
    # Load model weights
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    
    # Prediksi
    class_name, confidence, all_scores = predict_image(model, image_path)
    
    return class_name, confidence, all_scores

# Contoh penggunaan:
# model_path = 'models/best_vit_chicken_classifier.pth'
# image_path = 'path/to/test_image.jpg'
# class_name, confidence, all_scores = load_model_and_predict(model_path, image_path)